In [59]:
import glob
from multiprocessing import Pool
import sys
from nltk import TweetTokenizer
import nltk
import os
import re
import codecs
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.stem.snowball import EnglishStemmer
import numpy as np

## Loading Data

In [20]:
with open("en_train/english_train.text", 'r') as f:
    en_texts = [l.strip() for l in f]

In [61]:
with open('en_train/english_train.labels', 'r') as f:
    en_labels = [int(l.strip()) for l in f]

In [43]:
with open('test/english_test.text', 'r') as f:
    en_trial = [l.strip() for l in f]

In [62]:
with open('test/english_test.labels', 'r') as f:
    en_trial_labels = [int(l.strip()) for l in f]

In [37]:
with open("es_train/spanish_train.text", 'r') as f:
    es_texts = [l.strip() for l in f]

In [38]:
with open("es_train/spanish_train.labels", 'r') as f:
    es_labels = [l.strip().split() for l in f]

In [45]:
with open('test/spanish_test.text', 'r') as f:
    es_trial = [l.strip() for l in f]

In [46]:
with open('test/spanish_test.labels', 'r') as f:
    es_trial_labels = [int(l.strip()) for l in f]

In [40]:
with open("mapping/english_mapping.txt", 'r') as f:
    en_maps = [l.strip().split() for l in f]

In [41]:
with open("mapping/spanish_mapping.txt", 'r') as f:
    es_maps = [l.strip().split() for l in f]

In [2]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','<url>',tweet)
    tweet = re.sub('(\@[^\s]+)','<user>',tweet)
    try:
        tweet = tweet.decode('unicode_escape').encode('ascii','ignore')
    except:
        pass
    return tweet

In [10]:
def tokenize_tweets(filename, dest_folder):
    basename = os.path.basename(filename)
    dest = os.path.join(dest_folder, basename + '.tok')
    print("processing %s" % basename)
    tknzr = TweetTokenizer()
    with codecs.open(dest, 'w', "utf-8") as out_fs:
        with open(filename, 'r', encoding="utf-8") as in_fs:
            for line in in_fs:
#                 try:
#                     language, id, timestamp, username, tweet = line.strip().split('\t')
#                 except:
#                     print("could not parse line.")
#                     continue
#                 if language != 'en':
#                     continue
                tweet = tknzr.tokenize(line)
#                 if not 6 < len(tweet) < 110:
#                     continue
                tweet = preprocess_tweet(' '.join(tweet))
                out_fs.write(tweet+'\n')

In [18]:
p.set_options(p.OPT.URL, p.OPT.NUMBER, p.OPT.MENTION)
translator = str.maketrans("", "", punctuation)

In [165]:
def preproc_eng(texts):
    clear_texts = []
    for text in texts:
        # delete stop-words
        text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])
        print(text)
        # delete punctuation
        text = word_tokenize(text.translate(translator))     
        # stemming
#         text = [stemmer.stem(w) for w in text]
        # preprocessing as tweet
        text = p.clean(' '.join(text))
        clear_texts.append(text)
    return clear_texts

In [49]:
def preproc_es(texts):
    clear_texts = []
    for text in texts:
        # delete stop-words
        text = ' '.join([word for word in text.split() if word not in (stopwords.words('spanish'))])
        # delete punctuation
        text = word_tokenize(text.translate(translator))
        # stemming
#         text = [stemmer.stem(w) for w in text]
        # preprocessing as tweet
        text = p.clean(' '.join(text))
        clear_texts.append(text)
    return clear_texts

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weihaoran/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/weihaoran/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [166]:
texts_clear = preproc_eng(texts)

In [47]:
trial_clear = preproc_eng(en_trial)

In [50]:
es_text_clear = preproc_es(es_texts)

In [55]:
es_trial_clear = preproc_es(es_trial)

In [185]:
with open("en_tokenized/clear_Eng_train.txt",'r') as f:
    texts_clear = [l.strip() for l in f]

In [204]:
## get ride of odd symbol
for i in range(len(texts_clear)):
    texts_clear[i] = re.sub('[…·•’”“—➵・]', '', texts_clear[i])
    texts_clear[i] = texts_clear[i].lower()

In [205]:

for i in range(len(trial_clear)):
    trial_clear[i] = re.sub('[…·•’”“—➵・]', '', trial_clear[i]) 
    trial_clear[i] = trial_clear[i].lower()

In [234]:
for i in range(len(es_text_clear)):
    es_text_clear[i] = re.sub('[…·•’”“—➵・]', '', es_text_clear[i]) 
    es_text_clear[i] = es_text_clear[i].lower()

In [235]:
for i in range(len(es_trial_clear)):
    es_trial_clear[i] = re.sub('[…·•’”“—➵・]', '', es_trial_clear[i]) 
    es_trial_clear[i] = es_trial_clear[i].lower()

In [236]:
es_text_clear

['ahaa brevard county florida',
 'sleek black user toronto ontario',
 'sunny  s face asking me mean leaving abuela abuela ️',
 'with amy years ago tbt philadelphia museum art',
 'you know picked good husband brings slurpee blessed',
 'welsh siblings take unc university northern colorado unc',
 'i tv today lakers media day season im the',
 'new vlog chiefobivlogs this one i made jidennas the let out ft',
 'womancrushwednesday woman heart amp soul amp i thankful every moment have',
 'family golf outing user user user',
 'drink coffee make up girl just don  t push off',
 ' mixtape coming soon  user university windsor',
 'puppy love scooby parchita ️ mybabyisinlove love central park manhatan',
 'thanks tatianna coming ️ ️ ️ hair mywork woman blowdry haircut hairstylist',
 'the new omid salon cool shoreh cooler omidsalon shorehshoreh',
 'when highlight day kiss mom ️',
 'last nights memories captured escape fate i prevail',
 'when girl cant reach top shelf got tall guy loudoun county virgin

### Vocabulary Creation

In [207]:
vocab = set()
total_vocab = texts_clear + trial_clear
for line in total_vocab:
    for word in line.split():
        vocab.add(word)

In [291]:
es_vocab = set()
total_es = es_text_clear + es_trial_clear
for line in total_es:
    for word in line.split():
        es_vocab.add(word)

In [209]:
len(vocab)

103027

In [292]:
len(es_vocab)

38603

In [237]:
with codecs.open("en_tokenized/clear_Eng_train.txt",'w',"utf-8") as out_fs:
    for each in texts_clear:
        out_fs.write(each + "\n")

In [238]:
with codecs.open("es_tokenized/clear_Es_train.txt",'w',"utf-8") as out_fs:
    for each in es_text_clear:
        out_fs.write(each + "\n")

In [242]:
with codecs.open("es_tokenized/clear_Es_test.txt",'w',"utf-8") as out_fs:
    for each in es_trial_clear:
        out_fs.write(each + "\n")

In [240]:
with codecs.open("en_tokenized/clear_Eng_test.txt",'w',"utf-8") as out_fs:
    for each in trial_clear:
        out_fs.write(each + "\n")

## Sentence Vectorization

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

My baseline here is BagOfWords model and Tf-idf model.

### BagOfWords

#### English

In [228]:
cv_ = CountVectorizer(ngram_range=(-2,2), vocabulary=vocab_lst, min_df= 5)

Split train and dev dataset

In [229]:
en_train, en_dev = train_test_split(texts_clear, test_size = 0.2, shuffle=False)
en_y, en_dev_y = train_test_split(en_labels, test_size = 0.2, shuffle=False)

In [230]:
vocab_lst = list(vocab)

In [225]:
cv_.fit(raw_documents=vocab_lst)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [231]:
en_train_cv = cv_.fit_transform(en_train)
en_dev_cv = cv_.transform(en_dev)

In [232]:
en_train_cv

<72000x103027 sparse matrix of type '<class 'numpy.int64'>'
	with 546600 stored elements in Compressed Sparse Row format>

In [277]:
lr = LogisticRegression(random_state=14, dual = True)

In [278]:
lr.fit(en_train_cv, en_y)

/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=14, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [279]:
en_pred_dev = lr.predict(en_dev_cv)

In [280]:
en_pred_dev.dtype = np.int

In [281]:
np.savetxt('res_cv.txt', en_pred_dev, fmt='%d')
np.savetxt('goldres_cv.txt', np.array(en_dev_y), fmt='%d')

In [282]:
%run scorer_semeval18.py goldres_cv.txt res_cv.txt

Macro F-Score (official): 21.256
-----
Micro F-Score: 31.222
Precision: 31.222
Recall: 31.222


#### Spanish

In [294]:
cv_es = CountVectorizer(ngram_range=(-2,2), vocabulary=es_vocab)

In [ ]:
es_train, es_dev = train_test_split(es_text_clear, test_size = 0.2, shuffle=False)
es_y, es_dev_y = train_test_split(es_labels, test_size = 0.2, shuffle=False)

### Tf-idf

#### English

In [124]:
tf = TfidfVectorizer()

In [125]:
en_train_tf = tf.fit_transform(en_train)
en_dev_tf = tf.transform(en_dev)

In [141]:
Cs = np.logspace(0,1.5,20)

In [142]:
for C in Cs:
    print("C: " , C )
    lr = LogisticRegression(C=C, dual= True, random_state=14)
    lr.fit(en_train_tf, en_y)
    en_pred_train = lr.predict(en_dev_tf)
    en_pred_train.dtype = np.int
    np.savetxt('res.txt', en_pred_train, fmt='%d')
    np.savetxt('goldres.txt', np.array(en_dev_y), fmt='%d')
    %run scorer_semeval18.py goldres.txt res.txt
    print()

C:  1.0


/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Macro F-Score (official): 19.725
-----
Micro F-Score: 32.461
Precision: 32.461
Recall: 32.461

C:  1.1993539462092342
Macro F-Score (official): 20.046
-----
Micro F-Score: 32.533
Precision: 32.533
Recall: 32.533

C:  1.4384498882876628
Macro F-Score (official): 20.349
-----
Micro F-Score: 32.483
Precision: 32.483
Recall: 32.483

C:  1.7252105499420405
Macro F-Score (official): 20.83
-----
Micro F-Score: 32.55
Precision: 32.55
Recall: 32.55

C:  2.0691380811147897
Macro F-Score (official): 21.066
-----
Micro F-Score: 32.45
Precision: 32.45
Recall: 32.45

C:  2.481628922836826
Macro F-Score (official): 21.256
-----
Micro F-Score: 32.333
Precision: 32.333
Recall: 32.333

C:  2.976351441631318
Macro F-Score (official): 21.313
-----
Micro F-Score: 31.989
Precision: 31.989
Recall: 31.989

C:  3.5696988468260646
Macro F-Score (official): 21.571
-----
Micro F-Score: 31.778
Precision: 31.778
Recall: 31.778

C:  4.281332398719393
Macro F-Score (official): 21.752
-----
Micro F-Score: 31.533
Preci

/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Macro F-Score (official): 20.873
-----
Micro F-Score: 28.211
Precision: 28.211
Recall: 28.211

C:  21.983926488622892
Macro F-Score (official): 20.593
-----
Micro F-Score: 27.7
Precision: 27.7
Recall: 27.7

C:  26.366508987303583
Macro F-Score (official): 20.277
-----
Micro F-Score: 27.2
Precision: 27.2
Recall: 27.2

C:  31.622776601683793
Macro F-Score (official): 20.071
-----
Micro F-Score: 26.8
Precision: 26.8
Recall: 26.8



In [143]:
lr = LogisticRegression(C=4, dual= True, random_state=14)

In [144]:
lr.fit(en_train_tf, en_y)

/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=14, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [145]:
en_pred_train = lr.predict(en_dev_tf)
en_pred_train.dtype = np.int
np.savetxt('res.txt', en_pred_train, fmt='%d')
np.savetxt('goldres.txt', np.array(en_dev_y), fmt='%d')

In [152]:
lr = LogisticRegression(C=10.0, dual= True, solver="liblinear", random_state=14)

In [153]:
en_train_clear = tf.fit_transform(texts_clear)
en_test_clear = tf.transform(trial_clear)


In [154]:
lr.fit(en_train_clear, en_labels)

/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=10.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=14, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [155]:
res_en = lr.predict(en_test_clear)

In [156]:
res_en.dtype = np.int

In [157]:
np.savetxt("res.txt", res_en, fmt="%d")

In [158]:
%run scorer_semeval18.py test/english_test.labels res.txt

Macro F-Score (official): 21.684
-----
Micro F-Score: 30.52
Precision: 30.52
Recall: 30.52


#### Spanish

In [266]:
es_labels = np.array(es_labels).astype(int)

In [267]:
es_train, es_dev = train_test_split(es_text_clear, test_size = 0.2, shuffle=False)
es_y, es_dev_y = train_test_split(es_labels, test_size = 0.2, shuffle=False)

In [268]:
tf_es = TfidfVectorizer()

In [269]:
es_train_tf = tf.fit_transform(es_train)
es_dev_tf = tf.transform(es_dev)

In [270]:
for C in Cs:
    print("C: " , C )
    lr = LogisticRegression(C=C, dual= True, random_state=14)
    lr.fit(es_train_tf, es_y)
    es_pred_train = lr.predict(es_dev_tf)
    es_pred_train.dtype = np.int
    np.savetxt('res_es.txt', es_pred_train.astype(int), fmt='%d')
    np.savetxt('goldres_es.txt', np.array(es_dev_y).astype(int), fmt='%d')
    %run scorer_semeval18.py goldres_es.txt res_es.txt
    print()

C:  1.0


/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Macro F-Score (official): 7.665
-----
Micro F-Score: 28.237
Precision: 28.237
Recall: 28.237

C:  1.1993539462092342
Macro F-Score (official): 7.882
-----
Micro F-Score: 28.0
Precision: 28.0
Recall: 28.0

C:  1.4384498882876628
Macro F-Score (official): 8.212
-----
Micro F-Score: 28.026
Precision: 28.026
Recall: 28.026

C:  1.7252105499420405
Macro F-Score (official): 8.625
-----
Micro F-Score: 27.816
Precision: 27.816
Recall: 27.816

C:  2.0691380811147897
Macro F-Score (official): 8.849
-----
Micro F-Score: 27.342
Precision: 27.342
Recall: 27.342

C:  2.481628922836826
Macro F-Score (official): 9.227
-----
Micro F-Score: 26.895
Precision: 26.895
Recall: 26.895

C:  2.976351441631318
Macro F-Score (official): 9.337
-----
Micro F-Score: 26.5
Precision: 26.5
Recall: 26.5

C:  3.5696988468260646
Macro F-Score (official): 9.99
-----
Micro F-Score: 26.105
Precision: 26.105
Recall: 26.105

C:  4.281332398719393
Macro F-Score (official): 10.019
-----
Micro F-Score: 25.711
Precision: 25.711
R

/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Macro F-Score (official): 10.763
-----
Micro F-Score: 23.263
Precision: 23.263
Recall: 23.263

C:  26.366508987303583
Macro F-Score (official): 10.889
-----
Micro F-Score: 23.132
Precision: 23.132
Recall: 23.132

C:  31.622776601683793
Macro F-Score (official): 10.816
-----
Micro F-Score: 23.0
Precision: 23.0
Recall: 23.0



In [283]:
es_texts_tf = tf_es.fit_transform(es_text_clear)
es_test_tf = tf_es.transform(es_trial_clear)

In [284]:
lr = LogisticRegression(C=26.4, dual= True, random_state=14)
lr.fit(es_texts_tf, es_labels)
pred_es = lr.predict(es_test_tf)
pred_es = np.array(pred_es).astype(int)

/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/weihaoran/env/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [286]:
len(pred_es)

1000

In [288]:
es_test_tf.shape

(1000, 36801)

In [290]:
np.savetxt("res_es.txt", pred_es, fmt="%d")
%run scorer_semeval18.py test/spanish_test.labels res_es.txt

Macro F-Score (official): 11.804
-----
Micro F-Score: 25.7
Precision: 25.7
Recall: 25.7
